In [1]:
# Imports
import numpy as np
import seaborn as sns
from skmultiflow.drift_detection.adwin import ADWIN
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm 
import imageio.v2 as imageio
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.cm as cm
import matplotlib.animation as animation
import cv2
from time import sleep
import gc
from glob import glob
import joblib
from joblib import Parallel, delayed
import warnings 
warnings.filterwarnings("ignore")
import os
from random import sample, seed, randint, random, shuffle, choice, choices, uniform, gauss, triangular
from PIL import Image, ImageOps 
from pandas_datareader import data as pdr
import pandas as pd
import shutil
from collections import OrderedDict

In [2]:
def normalize_list(list:list):
        min_val = min(list)
        max_val = max(list)
        return [(x-min_val)/(max_val-min_val) for x in list]
def load_(filename):
        i = plt.imread(filename)
        return  i
def load_frames(i, h, w):
    return np.uint8(cv2.resize(imageio.imread(f"/tmp/{i}.jpg"), (h,w)))
def normalize_array(array:np.ndarray):
    min_val = np.min(array)
    max_val = np.max(array)
    return (array-min_val)/(max_val-min_val)
def split_list(list:list, n:int):
    return [list[i:i+n] for i in range(0, len(list), n)]
def plot_images(i, x, diffs, ground_truth, label_plot, preds, points):
        window = 50
        if i > 50: window = i - 50
        else: window = 0
        
        #plot preds
        fig = plt.figure()
        fig.set_size_inches(10, 10, forward=True)
        ax = fig.add_subplot(2,2,1)
        ax.set_title(f"prediction {window}:{i}")
        ax.imshow(preds, cmap='gray') 

        #plot diffs
        ax = fig.add_subplot(2,2,2)
        ax.set_title(f"diff {window}:{i}")
        ax.imshow(diffs, cmap='gray')

        #plot ground_truth
        ax = fig.add_subplot(2,2,4)
        ax.set_title(f"ground truth {window}:{i}")
        ax.imshow(ground_truth, cmap='gray')

        #plot anomaly score
        ax = fig.add_subplot(2,2,3)
        ax.set_title(f"anomaly score {window}:{i}")
        ax.plot(np.arange(len(x[window:i])), x[window:i], color='black', label='anomaly score', linewidth=0.6)
        ax.scatter(np.arange(len(x[window:i])), x[window:i], color='black', label='anomaly score', s=4)
        ax.scatter(np.arange(len(label_plot['x'][window:i])), label_plot['y'][window:i], color='red', marker='x', s=4)

        # find all search.points that are in the window
        matching = [(i,points[x]) for i, x in enumerate(np.arange(window, i)) if x in points.index]
        # plot these points with y-values = data y values
        ax.scatter([x[0] for x in matching], [x[1] for x in matching], label='Flagged points', color='purple', s=14, alpha=0.5, marker='o')


        ax.legend([metric])
        fig.tight_layout()
        fig.subplots_adjust(wspace=0, hspace=0)
        #set ax to show 50 x values
        ax.set_xlim(0, 50)
        ax.set_ylim(0, 1)
        fig.savefig(f"/tmp/{i}.jpg", bbox_inches='tight', pad_inches=0)
        
        fig.clear()
        plt.close(fig)
        del ax
        del window
        del fig
        return None
def copy_frames(folder, destination, keep_every):
            files = sorted(glob(os.path.join(folder, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            dest = os.path.join(destination, folder.split('/')[-1])
            for i, file in enumerate(files):
                if i % keep_every == 0:
                    shutil.copy(file, os.path.join(dest, os.path.basename(file)))
            # rename all files in the folder to be sequential, starting at 0
            files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            for i, file in enumerate(files):
                os.rename(file, os.path.join(dest, str(i).zfill(3)+'.jpg'))

            # remove frames until len(files) is a multiple of 5
            files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            while len(files) % 1 != 0:
                os.remove(files[-1])
                files = sorted(glob(os.path.join(dest, '*.jpg')), key=lambda x: int(x.split('/')[-1].split('.')[0]))
            del files, dest, folder, destination, keep_every

class anomaly():
    def __init__(self, method, metric, std_correction, dataset):
        self.method, self.metric, self.std_correction, self.dataset = method, metric, std_correction, dataset
    ##############################################################################
    def swap_test_folders(self, inference='01'):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.inference = inference
        #check if frames exists:
        if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'):
            !rm -r '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'

        if inference == 'all':
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'
        else:
            if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames'):
                !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/'$inference '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'$inference
        ##############################################################################
        frame_labels_bugs = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy', allow_pickle=True)
        # expand dims and save again
        #frame_labels_bugs = np.expand_dims(frame_labels_bugs, axis=0)
        frame_labels_bugs[0]
        #np.save('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/frame_labels_bugs.npy', frame_labels_bugs)
        if True:
            frame_labels_bugs = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy', allow_pickle=True)

            # list directories in /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/ using glob
            # glob returns a list of all files in the directory
            dd = glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/*')
            dd.sort()
            directories = [x.split('/')[-1] for x in dd if x.split('/')[-1].isdigit()]
            directories.sort()
            directories = {d:len(glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + d + '/*.jpg')) for d in directories if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + d)}
            directories = OrderedDict(sorted(directories.items(), key=lambda x: int(x[0])))
            print(list(directories))
            iterate = 0
            for folder, num_frames in directories.items():
                if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/' + folder):
                    !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/'$folder

                    print(folder, num_frames)
                if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT' + folder + '/frame_labels_bugs.npy'):
                    !mkdir '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT'$folder
                    # seperate the labels for each folder from the frame_labels_bugs
                    print(folder, num_frames)
                    labels = frame_labels_bugs[0][iterate:iterate+int(num_frames)]
                    iterate += num_frames
                    np.save('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT' + folder + '/frame_labels_bugs.npy', np.array(np.expand_dims(np.array(labels), axis=0)))
        if os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'):
            !rm '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'
        if inference == 'all':
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'
        else:
            !ln -s '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT'$inference'/frame_labels_bugs.npy' '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy'

        # #rename all folders with numeric names within '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/' from 01 to 100...
        if False: 
            for folder_path in glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/*'):
                if folder_path.split('/')[-1].isdigit():
                    new_folder_path = folder_path.split('/')
                    new_folder_path[-1] = str(int(new_folder_path[-1]) + 1).zfill(2)
                    new_folder_path = '/'.join(new_folder_path)
                    !mv $folder_path $new_folder_path

        if False: 
            for folder_path in glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/*'):
                if folder_path.split('/')[-1].isdigit():
                    new_folder_path = folder_path.split('/')
                    new_folder_path[-1] = str(int(new_folder_path[-1]) + 1).zfill(2)
                    new_folder_path = '/'.join(new_folder_path)
                    !mv $folder_path $new_folder_path

        # for len(dir_files) in /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/, ensure num is dividable by 5. if not, remove files until it is
        if False:######################################################3
            for folder_path in sorted(glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/*'), key=lambda x: int(x.split('/')[-1])):
                if folder_path.split('/')[-1].isdigit():
                    dir_files = sorted(glob(folder_path + '/*'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
                    if len(dir_files) % 5 != 0:
                        
                        # how many files do we need to remove?
                        num_to_remove = len(dir_files) % 5
                        #print(folder_path, len(dir_files), num_to_remove, dir_files[-1])

                        # remove the last num_to_remove files
                        for i in range(num_to_remove):
                            print (dir_files[-1])
                            name = dir_files[-1-i]
                            !rm -r $name

        test = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_bugs.npy', allow_pickle=True)
        print(test.shape)
        (sum(directories.values()))
    



        
        
    def rm_train_music_copy(self):
        if not os.path.exists('/tmp/empty'):
            os.makedirs('/tmp/empty')
        !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/VQ-VAE-Search-main/mel_specs_music/train/cl/
        !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/VQ-VAE-Search-main/mel_specs_music/test/cl/
    def cut_dataset(self, keep_every=10, ram=True, source_path='/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/full', destination = '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames', ):
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.ls_destination = destination
        self.source_path = source_path
        self.destination = destination
        self.keep_every = keep_every
        self.ram = ram
        if not os.path.exists('/tmp/empty'):
            os.makedirs('/tmp/empty')
        if self.ram:
            if not os.path.exists('/home/smoothjazzuser/Desktop/ram/frames/'):
                os.makedirs('/home/smoothjazzuser/Desktop/ram/frames/')
            self.destination = '/home/smoothjazzuser/Desktop/ram/frames'
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/Desktop/ram/frames/
            !rm -rf /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
        else: 
            !rsync -a --delete /tmp/empty/   /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/
            #del /frames/ folder
            !rm -rf /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
            #create new /frames/ folder
            !mkdir /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames
            print('Saving to RAM')
        !mkdir -p /tmp/empty

        self.folders_list = sorted(glob(os.path.join(self.source_path, '*')))
        print(len(self.folders_list))

        # create all desitnation folders, if they don't exist
        for folder in self.folders_list:
            if not os.path.exists(os.path.join(self.destination, folder.split('/')[-1])):
                os.makedirs(os.path.join(self.destination, folder.split('/')[-1]))

        

        c = Parallel(n_jobs=12)(delayed(copy_frames)(folder, self.destination, self.keep_every) for folder in self.folders_list)
        del c
        if ram:
            !ln -s /home/smoothjazzuser/Desktop/ram/frames/ /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/


    def analysis(self):
        # change working directory in jupyter notebook
        os.chdir("/home/smoothjazzuser/videogame-anomoly/")
        self.anomaly_score_total_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/anomaly_score_total_list.npy", allow_pickle=True)
        self.auc = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/auc.npy", allow_pickle=True)
        self.feature_distance_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/feature_distance_list.npy", allow_pickle=True)
        self.psnr_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/psnr_list.npy", allow_pickle=True)
        self.label_list = np.load(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_{self.dataset}.npy", allow_pickle=True).tolist()[0]
        self.x = self.anomaly_score_total_list
        self.name = self.key = list(np.array(self.psnr_list).reshape(1).tolist()[0].keys())[0]
        print('len x',len(self.x),"len y",len(self.label_list), 'auc:',self.auc, "key:", self.key)
        ############################################################################################################
        self.psnr = normalize_list(np.array(self.psnr_list).reshape(1).tolist()[0][self.name])
        self.feature_distance = normalize_list(self.feature_distance_list.tolist()[self.name])
        self.adjustment = 4 if self.method == 'pred' else 0
        self.x = self.x[self.adjustment:]
        self.label_list = self.label_list[self.adjustment:]
        self.psnr = self.psnr[self.adjustment:]
        self.feature_distance = self.feature_distance[self.adjustment:]
        self.x_shape = self.x.shape[0]
        self.start = 0
        self.end = int(self.x_shape)
        ############################################################################################################
        if self.metric == 'anomaly_score':
            self.x = self.x[self.start:self.end]
        elif self.metric == 'feature_distance':
            self.x = self.feature_distance[self.start:self.end]
        elif self.metric == 'psnr':
            self.x = self.psnr[self.start:self.end]

        self.label_list = self.label_list[self.start:self.end]
        ############################################################################################################
        self.label_points = {'x':[], 'y':[]}
        self.label_plot = {'x':[], 'y':[]}
        for i in range(len(self.label_list)):
            if self.label_list[i] == 1:
                self.label_points['x'].append(i)
                self.label_points['y'].append(self.x[i])
                
                self.label_plot['y'].append(self.x[i])
            else:
                self.label_plot['y'].append(0)
            self.label_plot['x'].append(i)
        ############################################################################################################
        self.adwin = ADWIN(.05)
        self.drift = {'x': [], 'y': []}
        self.xpos = 0
        for i in range(self.x_shape):  # number of frames in vide0
            self.g = self.adwin.add_element(self.x[i])
            if self.adwin.detected_change():
                self.drift['y'].append(self.x[i])
                self.drift['x'].append(self.xpos)
                self.adwin.reset()
            elif self.adwin.detected_warning_zone():
                self.drift['y'].append(self.x[i])
                self.drift['x'].append(self.xpos)
            self.xpos += 1
        ############################################################################################################
        # plot the changes
        # self.fig, self.ax = plt.subplots()
        # self.fig.set_size_inches(200.5, 7.5, forward=True)
        # plt.xticks(np.arange(0, self.end+1, 10.0))

        # self.ax.plot(np.arange(len(self.x)), self.x, color='black', label='anomaly score', linewidth=0.6)
        # self.ax.scatter(np.arange(len(self.x)), self.x, color='black', label='anomaly score', s=4)
        # self.ax.scatter(self.label_points['x'], self.label_points['y'], color='red', marker='x', s=10)
        # # ax.scatter(drift['x'], drift['y'],color='green', marker='|', s=1000000)
        # self.ax.legend([self.metric,self.metric,'ground_truth'])
        # plt.savefig(f"/home/smoothjazzuser/Desktop/{self.dataset}_{self.method}_{self.metric}_folder-{self.name}_std_correction-{self.std_correction}.svg")
        # plt.show()
        ############################################################################################################
        stock, start, end, x, y = False, False, False, self.x, self.label_points
        self.data = x
        self.data = self.normalize_data(self.data)
        self.label_points = y
        self.data = pd.Series(self.data)
        self.label_points = pd.DataFrame.from_dict(self.label_points, orient='index')
        self.label_points = self.label_points.transpose() 
        self.zeros = pd.DataFrame(np.zeros(self.data.shape[0]), columns=['zeros'])
    
        ############################################################################################################
        
        self.plot(window=10, alpha=0.5,  plots = ['bet_std_mean', 'std', 'data','mean'])
        self.show_plot()
        self.plot(window=10, alpha=0.5,  plots = ['bet_std_mean', 'std','mean'])
        self.show_plot()

        self.r = int((len(self.label_list)-1)/1)
        self.preds = sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/preds/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))
        self.diffs = sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{self.dataset}/{self.method}/log/diffs/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))
        self.ground_truth = []
        self.folders = sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/{self.dataset}/testing/frames/*/"), key=lambda x: int(x.split('/')[-2]))
        self.files = [sorted(glob(f"{file}*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0])) for file in self.folders]
        for folder in sorted(glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/{self.dataset}/testing/frames/*/"), key=lambda x: int(x.split('/')[-2])):
            for file in sorted(glob(f"{folder}*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0])):
                self.ground_truth.append(plt.imread(file))

        self.preds = [plt.imread(pred) for pred in self.preds]
        self.diffs = [plt.imread(diff) for diff in self.diffs]
        !rm -rf /tmp/*.jpg
        !rm -rf /tmp/*.jpg
        !rm -rf /tmp/*.svg
        #split r into 10 chunks
        self.rr = list(range(self.r))
        a = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plot_images)(i, self.x, self.diffs[i], self.ground_truth[i], self.label_plot, self.preds[i], self.points) for i in self.rr[0:int(len(self.rr)/2)])
        del a
        gc.collect()
        a = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plot_images)(i, self.x, self.diffs[i], self.ground_truth[i], self.label_plot, self.preds[i], self.points) for i in self.rr[int(len(self.rr)/2):-1])
        del a
        gc.collect()
        del self.preds
        del self.diffs
        del self.ground_truth
        gc.collect()
        self.a = imageio.imread(f"/tmp/0.jpg").shape
        self.h,self.w = self.a[0], self.a[1]
        self.frames = Parallel(n_jobs=12, backend='multiprocessing')(delayed(load_frames)(i, self.h, self.w) for i in tqdm(range(self.r-1))) 
        !rm -rf /tmp/*.jpg
        !rm -rf /tmp/*.jpg
        !rm -rf /tmp/*.svg
        #delete all variables except frames
        gc.collect()
        imageio.mimsave(f'/home/smoothjazzuser/Desktop/{self.dataset}_{self.method}_{self.metric}_folder-{self.name}_std_correction-{self.std_correction}.mp4', self.frames, fps=10)
        del self.frames
        gc.collect()
    def normalize_data(self, data):
        #return data
        return (data - data.min()) / (data.max() - data.min())
    def download_data(self, ticker, start_date, end_date):
        self.data = pdr.get_data_yahoo(ticker, start_date, end_date)
        self.data = self.data.dropna()
        print(self.data.head())
        #normalize between 0 and 1
        self.data = self.normalize_data(self.data['Adj Close'])
        return self.data
    def rolling_average(self, data, window):
        roll = data.rolling(window).mean()
        #normalize between 0 and 1
        roll = self.normalize_data(roll)
        # correct for window size shift
        #roll = roll.shift(-window)
        return  roll
    def rolling_std(self, data, window):
        roll = data.rolling(window).std()
        #normalize between 0 and 1
        roll = self.normalize_data(roll)
        # correct for window size shift
        #roll = roll.shift(-window)
        return roll
    def gradient_at_each_point(self, data, window=False):
        if window: 
            self.gradient = np.gradient(data.rolling(window))
        else:
            self.gradient = np.gradient(self.data)
        self.gradient = pd.Series(self.gradient)
        #return gradient
    def points_above_rollingaverage_and_bellow_rollingstd(self, data, window, rolling=False):
        if rolling:
            rolling_mean = self.rolling_average(data, window)
            self.gradient_at_each_point(rolling_mean, window)
            rolling_std = self.rolling_std(data, window)
            points = data[(data > rolling_mean) & (data < rolling_std) & (self.gradient <= self.zeros.zeros)]
        else:
            rolling_std = self.rolling_std(data, window)
            self.gradient_at_each_point(data, False)
            points = data[(data < rolling_std) & (self.gradient <= self.zeros.zeros)]
        return points
    def plot(self, window=20, alpha=0.5, plots = ['data', 'mean', 'std', 'exponential', 'troughs', 'bet_std_mean', 'gradient']):
        plt.figure(figsize=(int(len(self.data)/76*1.5), 7))

        if 'mean' in plots:
            rolling_mean = self.rolling_average(self.data, window)
            plt.plot(rolling_mean, label='Rolling Mean')
        if 'std' in plots:
            rolling_std = self.rolling_std(self.data, window)
            plt.plot(rolling_std, label='Rolling Std', color='green')
        if 'exponential' in plots:
            exponential_smoothing = self.exponential_smoothing(self.data, alpha)
            plt.plot(exponential_smoothing, label='Exponential Smoothing')
        if 'troughs' in plots:
            troughs = self.lines_at_troughs(self.data, window)
            plt.plot(troughs, label='Lines at Troughs')
        if 'bet_std_mean' in plots:
            self.points = self.points_above_rollingaverage_and_bellow_rollingstd(self.data, window, rolling=False)
            # point y values should equal to data y values
            plt.plot(self.points, 'ro', label='Flagged points', color='purple', markersize=10, alpha=0.5, marker='o')
            # plot these points with y-values = data y values
        if 'data' in plots:
            plt.plot(np.arange(len(self.data)), self.data, color='black', label='data', linewidth=0.6)
            plt.scatter(np.arange(len(self.data)), self.data, color='black', label='data', s=4)
            plt.scatter(self.label_points['x'], self.label_points['y'], color='red', marker='x', s=10)
        plt.legend(loc='upper left')
        #return all plots
        self.all_plots = plt.gcf()
        #del plot
        plt.close('all')
        return self.all_plots
    def show_plot(self):
        plt.show()
        return self.all_plots
    ###############################################################################

        
    
run = anomaly(method='recon', metric='anomaly_score', std_correction='false', dataset="bugs")

In [ ]:
run.cut_dataset()

In [ ]:
# analysis
run.analysis()

In [ ]:
run.rm_train_music_copy()

In [6]:
run.swap_test_folders('02')

['02']
(1, 1545)
